# Анализ базы данных книжного магазина
***

# Описание задачи

Задача — проанализировать базу данных недавно приобретенного крупного сервиса для чтения книг по подписке.

### Краткое описание данных

Таблица `books` - данные о книгах, `authors` - данные об авторах, `publishers` - данные об издательствах, `ratings` - данные о пользовательских оценках книг, `reviews` - данные о пользовательских обзорах на книги.

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

# Исследование

## 1. Импортируем библиотеки

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv(encoding='ascii')


True

## 2. Подключимся к базе данных

Устанавливаем параметры:

In [2]:
db_config = {'user': os.environ.get('user'), # имя пользователя
             'pwd': os.environ.get('pwd'), # пароль
             'host': os.environ.get('host'),
             'port': os.environ.get('port'), # порт подключения
             'db': os.environ.get('db')} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

Сохраняем коннектор:

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Зададим простую функцию для чтения данных:

In [4]:
def query_handler(query):
    return pd.io.sql.read_sql(query, con=engine)

## 3. Решение задач

Сначала выведем первые и последние строки главной таблицы:

In [5]:
query = '''
SELECT *
FROM books;
'''

In [6]:
query_handler(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


&#9889; **Вывод:** в базе данные о 1000 книг.

### - посчитайтем, сколько книг вышло после 1 января 2000 года

In [7]:
query = """
SELECT COUNT(books)
FROM books
WHERE publication_date > '2000-01-01';
"""

In [8]:
query_handler(query)

,count
0,819


&#9889; **Вывод:** после 1 января 2000 года вышло 819 книг.

### - посчитайтем для каждой книги количество обзоров и среднюю оценку

In [9]:
query = """
WITH mean_rating AS (
    SELECT book_id,
           ROUND(AVG(rating), 2) AS average_rating
    FROM ratings
    GROUP BY book_id),
reviews_counter AS (
    SELECT book_id,
           COUNT(review_id) AS reviews_count
    FROM reviews
    GROUP BY book_id)
SELECT books.book_id,
       books.title,
       mean_rating.average_rating,
       reviews_counter.reviews_count
FROM books
LEFT OUTER JOIN mean_rating ON mean_rating.book_id = books.book_id
LEFT OUTER JOIN reviews_counter ON reviews_counter.book_id = books.book_id
ORDER BY average_rating DESC, reviews_count DESC;
"""

In [10]:
query_handler(query)

,book_id,title,average_rating,reviews_count
0,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
1,17,A Dirty Job (Grim Reaper #1),5.00,4.0
2,553,School's Out—Forever (Maximum Ride #2),5.00,3.0
3,444,Moneyball: The Art of Winning an Unfair Game,5.00,3.0
4,516,Plum Island (John Corey #1),5.00,2.0
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,202,Drowning Ruth,2.00,3.0
997,371,Junky,2.00,2.0
998,316,His Excellency: George Washington,2.00,2.0


&#9889; **Вывод:** Есть книги без ревью. Про первую строчку ничего не могу сказать, а вот Манибол читал - хороший :)

### - определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query = """
SELECT publishers.publisher,
       COUNT(book_id) AS published_books
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE num_pages > 50
GROUP BY publishers.publisher_id
ORDER BY published_books DESC
LIMIT 1;
"""

In [12]:
query_handler(query)

,publisher,published_books
0,Penguin Books,42


&#9889; **Вывод:** издательство "Пингвин" выпустило больше всех книг толще 50 листов - 42 штуки.

### - определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками

In [13]:
query = """
WITH rating_agg AS (SELECT book_id,
                           ROUND(AVG(rating), 2) AS mean_rating,
                           COUNT(rating_id)
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) > 50),
authors_rating AS (SELECT author,
                          rating_agg.mean_rating AS mean_author_rating
                   FROM books
                   JOIN rating_agg ON rating_agg.book_id = books.book_id
                   JOIN authors ON authors.author_id = books.author_id)
SELECT author, 
       AVG(mean_author_rating) AS mean_author_rating
FROM authors_rating
GROUP BY author
ORDER BY mean_author_rating DESC
LIMIT 1;
"""

In [14]:
query_handler(query)

,author,mean_author_rating
0,J.K. Rowling/Mary GrandPré,4.285


&#9889; **Вывод:** Джоан Роулинг и иллюстратор книг о Гарри Поттере Мэри Гранпрэ занимают первое место со средней оценкой 4.28 из 5. Молодчинки.

### - посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [15]:
query = """
WITH most_active_users AS (SELECT ratings.username,
                                  COUNT(ratings.rating_id)
                           FROM ratings
                           GROUP BY ratings.username
                           HAVING COUNT(rating_id) > 50),
count_of_reviews AS (SELECT reviews.username,
                            COUNT(review_id) AS cnt
                     FROM reviews
                     JOIN most_active_users ON most_active_users.username = reviews.username
                     GROUP BY reviews.username)
SELECT AVG(count_of_reviews.cnt)
FROM count_of_reviews
"""

In [16]:
query_handler(query)

,avg
0,24.333333


&#9889; **Вывод:** среднее количество обзоров для активных пользователей - 24.

## Вывод

1. В базе данных содержится 1000 наблюдений о книгах: информация о книгах, издательствах, авторах, пользовательские обзоры и рейтинги книг. 
2. Исходя из представленных данных, после после 1 января 2000 года вышло 819 книг, при этом издательство "Пингвин" выпустило больше всех книг толще 50 листов - 42 штуки. В принципе, лидерство издательства "Пингвин" не особо удивительно - тем и знамениты.
3. У Джоан Роулинг и иллюстратора книг о Гарри Поттере Мэри Гранпрэ - самая высокая средняя оценка книг - 4.28 из 5.  
4. Среднее количество обзоров для активных пользователей - 24, однако их активности не хватает на всех - некоторые книги не удостаиваются ревью.
5. Принимая во внимание, что по описанию задачи база даных принадлежит крупному сервису для чтения книг по подписке - 1000 наименований представляется весьма скромным (для сравнения, Букмейт утверждает, что у них в каталоге 850 000 книг на 12 языках) и кого-то, возможно, крупно обманули. Впрочем, мало не значит плохо, предлагаю оседлать всеобщую тенденцию: "Книга - роскошь для взыскательных и солидных господ - только лучшее, тщательно отобранный базовый цивилизационный мимнимум для ваших пытливых умов, мои дорогие богатеи, прямиком в ваших айпэдах. Ждите этой осенью".